In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

# Load and preprocess Titanic (simplified)
data = pd.read_csv('../datasets/titanic.csv')
data = data[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']].dropna()
X = data.drop('Survived', axis=1)
y = data['Survived']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define k values to test
k_values = [1, 3, 5, 7, 10, 15, 20]

# Perform cross-validation (5-fold)
cv_scores = []
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_scores.append(scores.mean())
    print(f"k={k}, Mean Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")

# Find optimal k
optimal_k = k_values[np.argmax(cv_scores)]
print(f"Optimal k: {optimal_k}, Best Accuracy: {max(cv_scores):.4f}")

# Train final model with optimal k
final_knn = KNeighborsClassifier(n_neighbors=optimal_k)
final_knn.fit(X_train, y_train)

# or use grid search to automate finding k
param_grid = {'n_neighbors': [1, 3, 5, 7, 10, 15, 20]}
knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)
print(f"Optimal k: {grid.best_params_['n_neighbors']}, Best Accuracy: {grid.best_score_:.4f}")


# Evaluate on test set
y_pred = final_knn.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.4f}")